# Demo: Question Answering on your own data using Azure OpenAI service embeddings + Python SDK

## Summary

In this demo, we use the [Python OpenAI SDK](https://github.com/openai/openai-python) with the [Azure OpenAI service](https://learn.microsoft.com/azure/cognitive-services/openai/overview) to tailor the model to answer questions specifically about the [meals](https://us.pycon.org/2023/onsite/meal-ingredients/) planned at Pycon.

> Note: access to the Azure OpenAI service is by approval only. Please see [How do I get access to Azure OpenAI service?](https://learn.microsoft.com/azure/cognitive-services/openai/overview#how-do-i-get-access-to-azure-openai) for more information.

## Prequisites and install instructions

- Python 3.7+
- An Azure OpenAI service resource (or alternatively an OpenAI account)

To install the necessary requirements to run the demo, run:

`pip install -r requirements.txt`

## Authentication

In this demo, we will configure the library to use the Azure OpenAI service and authenticate using Azure Active Directory (AAD). Alternatively, you can use an API key (see [here](https://github.com/openai/openai-python#microsoft-azure-endpoints)).

In [2]:
import os
import openai
import azure.identity

openai.api_type = "azure_ad"  # using azure endpoints with AAD auth
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_version = "2023-03-15-preview"

credential = azure.identity.DefaultAzureCredential()
token = credential.get_token("https://cognitiveservices.azure.com/.default")

openai.api_key = token.token

## Code

In [ ]:
"""We start by scraping the meal table from the Pycon webpage and converting it into a Dataframe."""

import pandas as pd

# read the table from the Pycon meal webpage
html = pd.read_html('https://us.pycon.org/2023/onsite/meal-ingredients/')
df = html[0]

# remove all the empty rows at the bottom
df = df.dropna(how='all')
df = df.reset_index()

# fill in empty cells and group ingredients that fall to second row
df['OPTION'].ffill(inplace=True)
df['MENU ITEM'].ffill(inplace=True)
df['MEAL'].ffill(inplace=True)
df = df.dropna(how='any')
df = (df.groupby(df['MENU ITEM'].ne(df['MENU ITEM'].shift()).cumsum(), as_index=False)
   .agg({'OPTION':'first', 'MEAL': 'first', 'MENU ITEM': "first", 'INGREDIENTS': ' '.join})
  )

# group the row text together under a new text column
df["text"] = df.apply(lambda x: f"{x['OPTION']}, {x['MEAL']}, {x['MENU ITEM']}, {x['INGREDIENTS']}", axis=1)
df.to_csv('data.csv') 

In [3]:

"""4/19/23 note: The meal ingredients webpage changed structure so we need to load the previous data from the csv into a dataframe"""
import pandas as pd
df = pd.read_csv("data.csv")

In [4]:
"""The text is tokenized using `tiktoken` and the number of tokens per text chunk is saved to the Dataframe."""

import tiktoken

# Load the cl100k_base tokenizer which is designed to work with the ada-002 model
tokenizer = tiktoken.get_encoding("cl100k_base")

# Tokenize the text and save the number of tokens to a new column
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))

In [5]:
"""An embedding is generated for each text chunk. API calls are subject to rate limits so we use backoff library (as an example) to implement exponential backoff."""

import openai
import backoff

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embeddings(x):
    return openai.Embedding.create(input=x, deployment_id='text-embedding-ada-002-2')['data'][0]['embedding']

df['embeddings'] = df.text.apply(lambda x: get_embeddings(x))
df.to_csv('embeddings.csv')
df.head()

,Unnamed: 0,OPTION,MEAL,MENU ITEM,INGREDIENTS,text,n_tokens,embeddings
0,0,MEAT,Wednesday Tutorial Lunch,Greek Chicken Power Bowl,"Grilled Chicken, Mixed Greens, Cucumber, Tomat...","MEAT, Wednesday Tutorial Lunch, Greek Chicken ...",42,"[-0.011288567446172237, -0.014967902563512325,..."
1,1,MEAT,Thursday Tutorial Lunch,Beef Stir Fry,"Beef, Stir Fried Vegetables, Rice w/Peas & Car...","MEAT, Thursday Tutorial Lunch, Beef Stir Fry, ...",34,"[-0.004255026578903198, -0.02019380033016205, ..."
2,2,MEAT,Friday Conference Breakfast,Turkey Sausage & Egg Burrito,"Turkey Sausage, Egg, Cheese, Flour Tortilla Wrap","MEAT, Friday Conference Breakfast, Turkey Saus...",29,"[-0.015813421458005905, -0.015404226258397102,..."
3,3,MEAT,Friday Conference Lunch,Grilled Chicken Pasta Bowl,"Grilled Chicken, Pasta, Broccoli, Alfredo Sauce","MEAT, Friday Conference Lunch, Grilled Chicken...",25,"[-0.00466617988422513, -0.018455589190125465, ..."
4,4,MEAT,Saturday Conference Breakfast,Bacon & Egg English Muffin,"Bacon, Egg, Cheese, English Muffin","MEAT, Saturday Conference Breakfast, Bacon & E...",25,"[-0.006820477079600096, -0.014287382364273071,..."


In [6]:
"""Create context to give to the model to use when answering the question.

0. Ask a question
1. Generate embedding for the question
2. Find the text chunk from the given context (our meal data) that is most similar to the question using cosine similarity
3. Return that as the context for which the model should use to answer the question

Input:
> Context text
> Question
"""

from openai.embeddings_utils import distances_from_embeddings

def create_context(
    question, df, max_len
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, deployment_id='text-embedding-ada-002-2')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

In [7]:
"""The generative model is prompted with the question and the relevant text chunks as context. It will answer the question if the answer is found in the context."""

def ask_question(
    df,
    model="gpt-4",
    question="what is for lunch on wednesday?",
    max_len=1800,
    debug=False,
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
    )

    if debug:
        print("Context:\n" + context)
        print("\n\n")

    messages = [{"role": "system", "content": "Answer the question in your own words based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"}]
    messages.append({"role": "user", "content": f"Context: {context}\n\n---\n\nQuestion: {question}\nAnswer:"})
    response = openai.ChatCompletion.create(
        messages=messages,
        deployment_id=model,
    )
    answer = response['choices'][0]['message']['content'].strip()
    return answer


### Ask a question

In [8]:
ask_question(df, question="I hate butternut squash. What meals should I avoid?")

'You should avoid the Vegan Power Bowl, which contains Butternut Squash, during Wednesday Tutorial Lunch.'